# Solving Relativistic Fluid 2D

### Introduction
The goal of this notebook is to study turbulence from a perfect relativistic fluid. We consider the stress energy tensor for a perfect fluid in Minkowski space,

\begin{equation}
T_{ab} = p\eta_{ab} + (e+p)u_a u_b
\end{equation}

The equations of motion for such fluid are,

\begin{equation}
\label{EOM}
\partial_a T^{ab} = 0. \quad (1)
\end{equation}

Here we focus on a conformal fluid, that is the stress energy tensor is traceless,

$$T_a^a = 0 \to e = -2p.$$

To solve eq.1 numerically, we rewrite the EOM in conservative variables $U=(D,S^i)$ where,

$$\frac{D}{2} = T^{00} = \frac{e}{2}\left(-1+\frac{3}{W^2}\right)$$

$$\frac{S^i}{2} = T^{0i} = \frac{3e}{2}\frac{v^i}{W^2}$$

Where we have the Lorentz factor $W = \sqrt{1-v^iv_i}$. Now the EOM in conservative form,

$$\partial_t D + \partial_i S^i = 0$$
$$\partial_t S^i + \partial_j(e \delta^{ij} + S^i v^j) = 0$$

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.cm as cm

## Parameters

In [2]:
# Number of grid points
Nx = 201
Ny = Nx
Nt = 2000

# Discretisation parameters
cfl = 1.0
dx  = 10./(Nx)
dy  = 10./(Ny)
dt  = cfl*min(dx,dy)

# Spacial linspace
x = np.linspace(0,dx*Nx,Nx)
y = np.linspace(0,dy*Ny,Ny)

# physical parameters
gamma = 1.5

# Numerical parameters (floor)
epsilon = 1e-5
dissip = -0.01

In [3]:
# Variables to store solutions
v_x_old = np.zeros((Nx,Ny))

v_y_old = np.zeros((Nx,Ny))

rho_old = np.ones((Nx,Ny))

D_old = np.zeros((Nx,Ny))

s_x_old = np.zeros((Nx,Ny))

s_y_old = np.zeros((Nx,Ny))

## Variable conversion (C->P)

To switch from conservative to primitive variable we first note that,

$$S_iS^i/D^2 = 9(1-W^2)(3-W^2)^2$$

Now let $\Theta = S_iS^i/D^2$ and solving for $W^2$ we get,

$$W^2 = \frac{3}{2\Theta} \left(2 \Theta - 3 + \sqrt{9-8\Theta}\right)$$

With this,

$$e = \frac{D}{-1+3W^{-2}}$$

$$v^i = \frac{W^2 S^i}{3e}$$

In [4]:
def CtoP(D,Sx,Sy):
    
    Nx = S_x_old.shape[0]
    Nx = S_x_old.shape[1]
    prho = np.zeros((Nx,Ny))
    v_x  = np.zeros((Nx,Ny))
    v_y  = np.zeros((Nx,Ny))
    W2   = np.zeros((Nx,Ny))
    
    # Compute theta
    theta = (np.multiply(Sx,Sx)+np.multiply(Sy,Sy))/np.multiply(D,D)
    
    # Compute W^2
    W2 = (1.5/theta)*(2*theta-3.0+np.sqrt(9.0-8.0*theta))
    if np.any(np.isnan(W2)):
        print(np.amin(9.0-8.0*theta))
        index1 = np.unravel_index(W2.argmin(), W2.shape)[0]
        index2 = np.unravel_index(W2.argmin(), W2.shape)[1]
        print("Position: ",index1,index2)
        print("Theta: ", theta[index1,index2])
        print("Sx: ",Sx[index1,index2])
        print("Sy: ",Sy[index1,index2])
        print("D: ",D[index1,index2])
    for i in range(Nx):
        for j in range(Ny):
            if (theta[i,j] < epsilon**2):
                W2[i,j] = 1.0
        
    # Compute primitives
    prho = D/(-1+(3/W2))
    
    v_x = (np.multiply(W2,Sx))/(3*prho)
    v_y = (np.multiply(W2,Sy))/(3*prho)
    
    return prho, v_x, v_y

def PtoC(rho,v_x,v_y,Nx,Ny):
    
    D = np.zeros((Nx,Ny))
    S_x = np.zeros((Nx,Ny))
    S_y = np.zeros((Nx,Ny))
    
    # Compute W^2
    W2 = np.ones((Nx,Ny))-(np.multiply(v_x,v_x)+np.multiply(v_y,v_y))
    
    D   = np.multiply(rho,-np.ones((Nx,Ny)) + 3.0/W2)
    S_x = 3.0*v_x/W2
    S_y = 3.0*v_y/W2
    
    return D, S_x, S_y

## Initial Data

In [5]:
def gen_init(dx,dy,Nx,Ny,x,y,plot=False):
    # initial data assumes square grid
    v_x = np.zeros((Nx,Ny))
    v_y = np.zeros((Nx,Ny))
    
    S_x = np.zeros((Nx,Ny))
    S_y = np.zeros((Nx,Ny))
    
    D = np.zeros((Nx,Ny))
    
    # Density of 1 everywhere
    rho = np.ones((Nx,Ny))
    
    W = np.zeros((Nx,Ny))
    W2 = np.zeros((Nx,Ny))
    
    # Generate velocities
    for i in range(Nx):
        if(y[i]>3 and y[i]<7):
            v_x[:,i] = 0.5*np.sin((2*np.pi)*y[i])*(y[i]-3.0)**2*((y[i]-7.0)**2)/32
        else:
            # Set to floor
            v_x[:,i] = epsilon
        if(x[i]>3 and x[i] < 7):
            v_y[i,:] = 0.5*np.sin((0.5*np.pi)*x[i])*(x[i]-3.0)**2*((x[i]-7.0)**2)/32
        else:
            # Set to floor
            v_y[i,:] = epsilon

    # Compute conservative variables
    D, S_x, S_y = PtoC(rho,v_x,v_y,Nx,Ny)

    if plot:
        fig = plt.figure(figsize=(15,15))
        ax = fig.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(x, y)
        
        ax.plot_surface(X, Y, S_x)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('S_x')
        
        plt.show()
    
    return v_x, v_y, rho, D, S_x, S_y

In [6]:
# Generate initial data
v_x_old, v_y_old, rho_old, D_old, S_x_old, S_y_old = gen_init(dx,dy,Nx,Ny,x,y)

### Define finite difference fn

In [7]:
# 2nd order Central difference for 1st degree derivative 
def dudx(u,dx):
    
    du = np.zeros((len(u),len(u)))
    
    for i in range(1,len(u)-1):
        du[i,:] = (u[i+1,:]-u[i-1,:])/(2*dx)
        
    # Assume periodic BC
    du[0,:]  = (u[1,:]-u[-2,:])/(2*dx)
    du[-1,:] = (u[1,:]-u[-2,:])/(2*dx)
        
    return du

# 2nd order Central difference for 1st degree derivative 
def dudy(u,dy):
    
    du = np.zeros((len(u),len(u)))
    
    for i in range(1,len(u)-1):
        du[:,i] = (u[:,i+1]-u[:,i-1])/(2*dy)
    
    # Assume periodic BC
    du[:,0]  = (u[:,1]-u[:,-2])/(2*dy)
    du[:,-1] = (u[:,1]-u[:,-2])/(2*dy)
        
    return du

# Need some dissapation
def dissip_u(u,dx):
    du = np.zeros((len(u),len(u)))
    du_temp = np.zeros((len(u),len(u)))
    for i in range(len(u)):
        du[:,i] = dissip4(u[:,i])
    for j in range(len(u)):
        du_temp[j,:] = dissip4(u[j,:])
    
    du = (du + du_temp)/dx
    
    return du

def dissip4(u):
    du = np.zeros(len(u))
    N = len(u)-1
    for i in range(2,N-1):
        du[i] = u[i+2]+u[i-2]-4*(u[i+1]+u[i-1])+6*u[i]
        
    du[1] = u[3]+u[-1]-4*(u[2]+u[0])+6*u[1]
    du[0] = u[2]+u[-2]-4*(u[1]+u[-1])+6*u[0]
    du[-1] = du[0]
    du[-2] = u[0]+u[-4]-4*(u[-1]+u[-3])+6*u[-2]
    return du

### Define fluxes

In [8]:
def floor(u,Nx):
    for i in range(Nx):
        for j in range(Nx):
            if u[i,j] < epsilon:
                u[i,j] = epsilon
    return u

def fluxes(rho,v_x,v_y,D,S_x,S_y):
    
    Fx_rho = S_x
    Fy_rho = S_y
    
    Fx_Sx = rho+np.multiply(S_x,v_x)
    Fy_Sx = np.multiply(S_x,v_y)
    
    Fx_Sy = np.multiply(S_y,v_x)
    Fy_Sy = rho+np.multiply(S_y,v_y)
    
    return Fx_rho, Fy_rho, Fx_Sx, Fy_Sx, Fx_Sy, Fy_Sy

def RHS(Z):
    
    D   = Z[0]
    S_x = Z[1]
    S_y = Z[2]
    
    D = floor(D,Nx)
    
    rho, v_x, v_y =  CtoP(D,S_x,S_y)
    
    # Compute fluxes
    Fx_rho, Fy_rho, Fx_Sx, Fy_Sx, Fx_Sy, Fy_Sy = fluxes(rho,v_x,v_y,D,S_x,S_y)
        
    # Compute derivatives
    dxFx_rho = dudx(Fx_rho,dx)
    dyFy_rho = dudy(Fy_rho,dx)
    
    dxFx_Sx  = dudx(Fx_Sx,dx)
    dyFy_Sx  = dudy(Fy_Sx,dx)
    
    dxFx_Sy  = dudx(Fx_Sy,dx)
    dyFy_Sy  = dudy(Fy_Sy,dx)
    
    dD   = (-dxFx_rho-dyFy_rho) + dissip*dissip_u(D,dx)
    dS_x = (-dxFx_Sx-dyFy_Sx) + dissip*dissip_u(S_x,dx)
    dS_y = (-dxFx_Sy-dyFy_Sy) + dissip*dissip_u(S_y,dx)
        
    del dxFx_rho,dyFy_rho,dxFx_Sx,dyFy_Sx,dxFx_Sy,dyFy_Sy
    return np.array([dD, dS_x, dS_y])

### Define RK4

In [9]:
def rk4_step(f, y, h):
    
    k1 = h*f(y)
    k2 = h*f(y + 0.5 * k1)
    k3 = h*f(y + 0.5 * k2)
    k4 = h*f(y + k3)
    
    new_data = y + (k1 + 2*k2 + 2*k3 + k4) / 6
    del k1,k2,k3,k4
    return new_data

### Evolve

In [ ]:
# Create Z vector
Z_old = np.array([D_old,S_x_old,S_y_old])
t = [0]
X, Y = np.meshgrid(x, y)

# Save D figure
figD, axD = plt.subplots(figsize=(15,15))
imD = axD.imshow(Z_old[0], interpolation='bilinear', 
origin='lower', extent=[min(x), max(x), min(y), max(y)],cmap=cm.RdYlGn,
              vmax=abs(Z_old[0]).max(), vmin=abs(Z_old[0]).min())

figSx, axSx = plt.subplots(figsize=(15,15))
imSx = axSx.imshow(Z_old[1], interpolation='bilinear', 
origin='lower', extent=[min(x), max(x), min(y), max(y)],cmap=cm.RdYlGn,
              vmax=abs(Z_old[1]).max(), vmin=abs(Z_old[1]).min())

figSy, axSy = plt.subplots(figsize=(15,15))
imSy = axSy.imshow(Z_old[2], interpolation='bilinear', 
origin='lower', extent=[min(x), max(x), min(y), max(y)],cmap=cm.RdYlGn,
              vmax=abs(Z_old[2]).max(), vmin=abs(Z_old[2]).min())

figD.savefig("D/D" + str(0) + ".png")
figSx.savefig("Sx/Sx" + str(0) + ".png")
figSy.savefig("Sy/Sy" + str(0) + ".png")

for i in range(1,Nt+1):
    Z_new = rk4_step(RHS,Z_old,dt)
    t.append(t[0]+i*dt)
    
    print(i)
    
    # Save D figure
    imD = axD.imshow(Z_old[0], interpolation='bilinear', origin='lower', extent=[min(x), max(x), min(y), max(y)],cmap=cm.RdYlGn,vmax=abs(Z_old[0]).max(), vmin=abs(Z_old[0]).min())
    figD.savefig("D/D" + str(int(i/10)) + ".png")
    
    # Save Sx figure
    imSx = axSx.imshow(Z_old[1], interpolation='bilinear', 
               origin='lower', extent=[min(x), max(x), min(y), max(y)],cmap=cm.RdYlGn,
               vmax=abs(Z_old[1]).max(), vmin=abs(Z_old[1]).min())
    figSx.savefig("Sx" + str(i) + ".png")
        
    # Save Sy figure
    imSy = axSy.imshow(Z_old[2], interpolation='bilinear', 
               origin='lower', extent=[min(x), max(x), min(y), max(y)],cmap=cm.RdYlGn,
               vmax=abs(Z_old[2]).max(), vmin=abs(Z_old[2]).min())
    figSy.savefig("Sy" + str(i) + ".png")
        
    Z_old = Z_new

1
2
3
4
5
